<a href="https://colab.research.google.com/github/Tixonmavrin/Zindi-Zimnat-Insurance-Recommendation-Challenge/blob/master/Baseline1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Zimnat Insurance Recommendation Challenge
Can you predict which insurance products existing clients will want next?
###Baseline with catboost

### Get data

In [19]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy
import sklearn

import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
from sklearn import preprocessing
from sklearn import metrics
import numpy as np
import pandas as pd
from time import time
import pprint
import joblib
from xgboost.sklearn import XGBClassifier 
import warnings
warnings.filterwarnings("ignore")

# Classifiers
from catboost import CatBoostClassifier

# Model selection
from sklearn.model_selection import StratifiedKFold

# Metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer

# Skopt functions
from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper, VerboseCallback, DeltaXStopper
from skopt.space import Real, Categorical, Integer
from time import time

In [49]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
submission = pd.read_csv('SampleSubmission.csv')

### New format

In [50]:
X_train = []
X_train_columns = train.columns
c = 0
for v in train.values:
    info = v[:8]
    binary = v[8:]
    index = [k for k, i in enumerate(binary) if i == 1]
    for i in index:
        c+=1
        for k in range(len(binary)):
            if k == i:
                binary_transformed = list(copy.copy(binary))
                binary_transformed[i] = 0
                X_train.append(list(info) + binary_transformed + [X_train_columns[8+k]] + [c])

X_train = pd.DataFrame(X_train)
X_train.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_pred', 'ID2']

In [51]:
X_test = []
true_values = []
c = 0
for v in test.values:
    c += 1
    info = v[:8]
    binary = v[8:]
    index = [k for k, i in enumerate(binary) if i == 1]
    X_test.append(list(info) + list(binary) + [c])
    for k in test.columns[8:][index]:
        true_values.append(v[0] + ' X ' + k)

X_test = pd.DataFrame(X_test)
X_test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'ID2']

### Transform data

In [52]:
features_train = []
features_test = []
columns = []

append_features = ['P5DA', 'RIBP', '8NN1', '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 
'N2MW', 'AHXO','BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 
'ECY3', 'ID', 'ID2', 'join_date', 'sex', 'marital_status', 'branch_code', 'occupation_code', 'occupation_category_code',
'birth_year']
for v in append_features:
    features_train.append(X_train[v].values.reshape(-1, 1))
    features_test.append(X_test[v].values.reshape(-1, 1))
    columns.append(np.array([v]))

y_train = X_train[['product_pred']]

In [53]:
features_train = np.concatenate(features_train, axis=1)
features_test = np.concatenate(features_test, axis=1)
columns = np.concatenate(np.array(columns))

X_train = pd.DataFrame(features_train)
X_train.columns = columns
X_test = pd.DataFrame(features_test)
X_test.columns = columns

### New features

In [54]:
X_train['date1'] = X_train['join_date'].apply(lambda x: int(x.split('/')[0]) if (x == x) else np.nan)
X_train['date2'] = X_train['join_date'].apply(lambda x: int(x.split('/')[1]) if (x == x) else np.nan)
X_train['date3'] = X_train['join_date'].apply(lambda x: int(x.split('/')[2]) if (x == x) else np.nan)
X_train.drop('join_date', axis=1, inplace=True)

X_test['date1'] = X_test['join_date'].apply(lambda x: int(x.split('/')[0]) if (x == x) else np.nan)
X_test['date2'] = X_test['join_date'].apply(lambda x: int(x.split('/')[1]) if (x == x) else np.nan)
X_test['date3'] = X_test['join_date'].apply(lambda x: int(x.split('/')[2]) if (x == x) else np.nan)
X_test.drop('join_date', axis=1, inplace=True)

X_train['date_diff'] = X_train['date3'] - X_train['birth_year']
X_test['date_diff'] = X_test['date3'] - X_test['birth_year']

### Change types

In [55]:
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)
y_train = y_train.fillna(0)

In [56]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data = X_train.append(X_test)
for v in ['sex', 'marital_status', 'branch_code', 'occupation_code', 'occupation_category_code',]:
    data.loc[:,v] = le.fit_transform(data.loc[:,v])
X_train = data[:X_train.shape[0]]
X_test = data[-X_test.shape[0]:]

In [57]:
le.fit(y_train.iloc[:,0])
y_train = pd.DataFrame(le.transform(y_train.iloc[:,0]))
y_train.columns = ['target']

In [58]:
pd.set_option('display.max_columns', None)
train.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,4WKQSBB,1/2/2019,F,M,1987,1X1H,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
1,CP5S02H,1/6/2019,F,M,1981,UAOD,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
2,2YKDILJ,1/6/2013,M,U,1991,748L,QZYX,90QI,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3,2S9E81J,1/8/2019,M,M,1990,1X1H,BP09,56SI,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
4,BHDYVFT,1/8/2019,M,M,1990,748L,NO3L,T4MS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0


In [59]:
train['join_date'] = pd.to_datetime(train['join_date'])
train['year'] = train['join_date'].dt.year
train['month'] = train['join_date'].dt.month
train['day'] = train['join_date'].dt.day

In [60]:
test['join_date'] = pd.to_datetime(test['join_date'])
test['year'] = test['join_date'].dt.year
test['month'] = test['join_date'].dt.month
test['day'] = test['join_date'].dt.day

In [61]:
X_train['year'] = train['year']
X_train['month'] = train['month']
X_train['day'] = train['day']
pd.set_option('display.max_columns', None)
X_train.head()

,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3,ID,ID2,sex,marital_status,branch_code,occupation_code,occupation_category_code,birth_year,date1,date2,date3,date_diff,year,month,day
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4WKQSBB,1,0,2,0,19,5,1987,1.0,2.0,2019.0,32.0,2019.0,1.0,2.0
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4WKQSBB,2,0,2,0,19,5,1987,1.0,2.0,2019.0,32.0,2019.0,1.0,6.0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,CP5S02H,3,0,2,11,19,5,1981,1.0,6.0,2019.0,38.0,2013.0,1.0,6.0
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,CP5S02H,4,0,2,11,19,5,1981,1.0,6.0,2019.0,38.0,2019.0,1.0,8.0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,2YKDILJ,5,1,6,3,185,1,1991,1.0,6.0,2013.0,22.0,2019.0,1.0,8.0


In [62]:
X_test['year'] = test['year']
X_test['month'] = test['month']
X_test['day'] = test['day']
pd.set_option('display.max_columns', None)
X_test.head()

,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3,ID,ID2,sex,marital_status,branch_code,occupation_code,occupation_category_code,birth_year,date1,date2,date3,date_diff,year,month,day
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,F86J5PC,1,1,2,4,96,1,1984,1.0,12.0,2018.0,34.0,2018.0,1.0,12.0
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,H6141K3,2,1,2,0,128,1,1996,1.0,10.0,2019.0,23.0,2019.0,1.0,10.0
2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,RBAYUXZ,3,0,7,11,19,5,1968,1.0,1.0,2020.0,52.0,2020.0,1.0,1.0
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,KCBILBQ,4,1,2,4,19,5,1989,1.0,2.0,2019.0,30.0,2019.0,1.0,2.0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,LSEC1ZJ,5,0,2,11,3,5,1982,1.0,2.0,2020.0,38.0,2020.0,1.0,2.0


### Model

In [ ]:
# model = CatBoostClassifier()
# parameters = {'depth'         : sp_randInt(4, 10),
#               'learning_rate' : sp_randFloat(),
#               'iterations'    : sp_randInt(10, 100)
#              }

# randm = RandomizedSearchCV(estimator=model, param_distributions = parameters,
#                            cv = 2, n_iter = 10, n_jobs=-1)
# randm.fit(X_train.drop(columns=['ID', 'ID2']), y_train, cat_features=['sex','marital_status','branch_code','occupation_code','occupation_category_code'])

# # Results from Random Search
# print("\n========================================================")
# print(" Results from Random Search " )
# print("========================================================")
# print("\n The best estimator across ALL searched params:\n",randm.best_estimator_)
# print("\n The best score across ALL searched params:\n",randm.best_score_)
# print("\n The best parameters across ALL searched params:\n",randm.best_params_)
# print("\n ========================================================")
# # model = CatBoostClassifier()
# # model.fit(X_train.drop(columns=['ID', 'ID2']), y_train, cat_features=['sex','marital_status','branch_code','occupation_code','occupation_category_code'])

In [ ]:
# model = CatBoostClassifier(depth= 7, iterations= 79, learning_rate= 0.24800286296598373)
# model.fit(X_train.drop(columns=['ID', 'ID2']), y_train, cat_features=['sex','marital_status','branch_code','occupation_code','occupation_category_code'])

In [ ]:
# def report_perf(optimizer, X, y, title, callbacks=None):
#     """
#     A wrapper for measuring time and performances of different optmizers
    
#     optimizer = a sklearn or a skopt optimizer
#     X = the training set 
#     y = our target
#     title = a string label for the experiment
#     """
#     start = time()
#     if callbacks:
#         optimizer.fit(X, y, callback=callbacks, cat_features=['sex','marital_status','branch_code','occupation_code','occupation_category_code'])
#     else:
#         optimizer.fit(X, y, cat_features=['sex','marital_status','branch_code','occupation_code','occupation_category_code'])
#     d=pd.DataFrame(optimizer.cv_results_)
#     best_score = optimizer.best_score_
#     best_score_std = d.iloc[optimizer.best_index_].std_test_score
#     best_params = optimizer.best_params_
#     print((title + " took %.2f seconds,  candidates checked: %d, best CV score: %.3f "
#            +u"\u00B1"+" %.3f") % (time() - start, 
#                                   len(optimizer.cv_results_['params']),
#                                   best_score,
#                                   best_score_std))    
#     print('Best parameters:')
#     pprint.pprint(best_params)
#     print()
#     return best_params

In [ ]:
# roc_auc = make_scorer(roc_auc_score, greater_is_better=True, needs_threshold=True)
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [ ]:
# clf = CatBoostClassifier(cat_features=['sex','marital_status','branch_code','occupation_code','occupation_category_code'])

In [ ]:
# search_spaces = {'iterations': Integer(10, 1000),
#                  'depth': Integer(1, 8),
#                  'learning_rate': Real(0.01, 1.0, 'log-uniform'),
#                  'random_strength': Real(1e-9, 10, 'log-uniform'),
#                  'bagging_temperature': Real(0.0, 1.0),
#                  'border_count': Integer(1, 255),
#                  'l2_leaf_reg': Integer(2, 30),
#                  'scale_pos_weight':Real(0.01, 1.0, 'uniform')}

In [ ]:
# # Setting up BayesSearchCV
# opt = BayesSearchCV(clf,
#                     search_spaces,
#                     scoring=roc_auc,
#                     cv=skf,
#                     n_iter=100,
#                     n_jobs=1,  # use just 1 job with CatBoost in order to avoid segmentation fault
#                     return_train_score=False,
#                     refit=True,
#                     optimizer_kwargs={'base_estimator': 'GP'},
#                     random_state=42)

In [ ]:
### Random Forest

# rf = RandomForestClassifier(
#     n_estimators=421, 
#     max_depth=15,
#     max_features=3,
#     min_samples_split=8, 
#     class_weight="balanced",
#     bootstrap=True,
#     criterion='entropy',
#     random_state=100
#     )

# rf.fit(X_train.drop(columns=['ID', 'ID2']), y_train)
# preds_rf = rf.predict_proba(X_test.drop(columns=['ID','ID2'], axis=1))
# y_test = pd.DataFrame(preds_rf)
# y_test.columns = le.inverse_transform(y_test.columns)

In [ ]:
import numpy as np
from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
pbounds = {
    'learning_rate': (0.01, 1.0),
    'n_estimators': (100, 1000),
    'max_depth': (3,10),
    'subsample': (1.0, 1.0),  # Change for big datasets
    'colsample': (1.0, 1.0),  # Change for datasets with lots of features
    'gamma': (0, 5),
}
def xgboost_hyper_param(learning_rate,
                        n_estimators,
                        max_depth,
                        subsample,
                        colsample,
                        gamma):
    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    clf = XGBClassifier(
        max_depth=max_depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        gamma=gamma,
        )
    return np.mean(cross_val_score(clf, X_train.drop(columns=['ID', 'ID2'], axis=1), y_train, cv=3))
optimizer = BayesianOptimization(
    f=xgboost_hyper_param,
    pbounds=pbounds,
    random_state=1,
)

optimizer.maximize(init_points=3,n_iter=10)

|   iter    |  target   | colsample |   gamma   | learni... | max_depth | n_esti... | subsample |
-------------------------------------------------------------------------------------------------


In [ ]:
## XGboost

xgb_cl = XGBClassifier(learning_rate=0.1675, max_depth=10, gamma=1.976, n_estimators=550)
xgb_cl.fit(X_train.drop(columns=['ID', 'ID2']),y_train)
preds_xgb = xgb_cl.predict_proba(X_test.drop(columns=['ID','ID2'], axis=1))
y_test = pd.DataFrame(preds_xgb)
y_test.columns = le.inverse_transform(y_test.columns)

### Submit

In [14]:
answer_mass = []
for i in range(X_test.shape[0]):
    id = X_test['ID'].iloc[i]
    for c in y_test.columns:
        answer_mass.append([id + ' X ' + c, y_test[c].iloc[i]])

df_answer = pd.DataFrame(answer_mass)
df_answer.columns = ['ID X PCODE', 'Label']
for i in range(df_answer.shape[0]):
    if df_answer['ID X PCODE'].iloc[i] in true_values:
        df_answer['Label'].iloc[i] = 1.0

In [15]:
df_answer.reset_index(drop=True, inplace=True)
df_answer.to_csv('submission12.csv', index=False)

Submit